# Multiple Hubs

Here we show how to manually combine hub objects so that they are nested in larger hubs.  
In the [Multiple Hubs](MultipleHubs.ipynb) notebook this is done for us using the `multiple_hubs` function.

In [1]:
from pyehub.energy_hub.ehub_model import EHubModel
from pyehub.energy_hub.utils import constraint
from pyehub import network_to_request_format
from pyehub import multiple_hubs
from pyehub.multiple_hubs import multiple_hubs
from pyehub.multiple_hubs import NetworkModel
from pyehub.energy_hub.input_data import InputData
import pylp
import numpy as np
from pylp import RealVariable
from pprint import pprint as pp

### Network Input as Request JSON

The data for the links can be supplied to the multiple hubs in either an excel file or in the network request format.

In [2]:
network_json = network_to_request_format.convert('network.xlsx')
pp(network_json)

{'capacities': [{'bounds': {'lower': 0, 'upper': 9999},
                 'name': 'capacity0',
                 'type': 'Continuous',
                 'units': ''},
                {'bounds': {'lower': 0, 'upper': 9999},
                 'name': 'capacity1',
                 'type': 'Continuous',
                 'units': ''},
                {'bounds': {'lower': 0, 'upper': 9999},
                 'name': 'capacity2',
                 'type': 'Continuous',
                 'units': ''},
                {'bounds': {'lower': 0, 'upper': 9999},
                 'name': 'capacity3',
                 'type': 'Continuous',
                 'units': ''}],
 'links': [{'capacity': 'capacity0',
            'end_id': 1,
            'id': 0,
            'length': 0.5,
            'operating_temp': '',
            'reactance': '',
            'resistance': '',
            'start_id': 0,
            'total_pressure_loss': '',
            'total_thermal_loss': 0.99,
            'type': '',
          

### Hub Inputs

When running the multiple hubs script the hubs must be in some pattern with incrementing values starting at 1. In this instance of running the code manually the hubs can be listed out one by one with any name. The individual hubs in the network use a custom version of the EHub model that adds links and applies custom constraints. 

In [3]:
network = network_json
hub1 = NetworkModel(excel='hub__1.xlsx', network_request=network, hub_id=0)
hub2 = NetworkModel(excel='hub__2.xlsx', network_request=network, hub_id=1)
hub3 = NetworkModel(excel='hub__3.xlsx', network_request=network, hub_id=2)
hub4 = NetworkModel(excel='hub__4.xlsx', network_request=network, hub_id=3)

### Tying the hub ids to their links in the network data
The link information is stored in a seperate network excel or network request file. Each link has a starting and ending point id which connects to the assosciated hub in the list id.

In [4]:
_net_data = InputData(network)
LINK_THERMAL_LOSS = _net_data.link_thermal_loss
connections = []
for i in _net_data.links_ids:
    start = _net_data.link_start[i]
    end = _net_data.link_end[i]

    connections.insert(i, (start, end))
print(connections)

[(0, 1), (1, 2), (2, 0), (1, 3)]


### Creating a list of all the constraints from all the hubs
For creating the larger problem to be solved with the model all the individual constraints for each Hub are all appended to one list.

In [5]:
hubs = [hub1, hub2, hub3, hub4]
constraints = []
for hub in hubs:
    hub.recompile()
    for constr in hub.constraints:
        constraints.append(constr)

### Network specific constraints
On top of the specific Hub constraints there is also a link capacity and energy balancing network constraint that is applied once for the entire network.

In [6]:
@constraint()
def link_capacity_constraint(link, hub, i):
    """
    Constraint for the flow in the links.
    """
    for flow in link:
        yield flow >= 0
        yield flow <= hub.link_capacities[i]
        
@constraint()
def network_constraint(hub, link_end, link_start):
    """
    Yields the constraints that allow a network connection between two hubs.

    Args:
        hub: The hub
        link_end: all the links that the hub ends at
        link_start: all the links that the hub start at
    Yields:
       A network energy balanced constraints for each hub
    """

    for t in hub.time:
        link_starting = []
        link_ending = []

        for i in range(len(link_start)):
            link_starting.append(link_start[i][t])

        for i in range(len(link_end)):
            link_ending.append(link_end[i][t])

        yield ((hub.energy_exported[t]['Net_export'] - hub.energy_imported[t]['Net_import'] + sum(link_ending)
                - sum(link_starting)) == 0)


### Energy flow Variables created for each connection
Each link has an energy_flow variable that is part of the model's solution.

In [7]:
energy_flow = {
    t: {con: RealVariable() for con in range(len(connections))}
    for t in hubs[0].time
}
links = []
for i in range(len(connections)):
    hub = hubs[0]
    flow = []
    for t in hub.time:
        flow.append(energy_flow[t][i])
    links.insert(i, flow)
links

[[<pylp.variable.RealVariable at 0x7f48854a14a8>,

### Applying link capacity constraint to each link

In [8]:
for hub in hubs:
    for i, link in enumerate(links):
        for c in link_capacity_constraint(link, hub, i):
            constraints.append(c)

### Applying energy balance constraint to each link

In [9]:
for k, hub in enumerate(hubs):
        # New list for all the links that the hub starts at
        link_start = []
        # New list for all the links that the hub ends at
        link_end = []

        for i in range(len(connections)):
            if connections[i][0] == k:
                link_start.append(links[i])
            # Searching for the links that the hubs ends at but are not power flow
            if connections[i][1] == k:
                # Multiplying by thermal loss to account for the thermal loss in the link
                links[i] = np.array(links[i])
                links[i] = links[i] * LINK_THERMAL_LOSS[i]
                link_end.append(links[i])
                
        # Creating the network constraints for the hubs:
        # Energy balancing for none powerflow links
        for c in network_constraint(hub, link_end, link_start):
            constraints.append(c)

### Combining the objectives of all the hubs and Solving
All the objectives and constraints are sent to pylp to form the MILP problem to be solved by the solver. The solution is then found dispersed in the solution dictionaries of the hubs.

In [10]:
objective = hubs[0].objective
for hub in hubs[1:]:
    objective += hub.objective

# Now solve this model.
status = pylp.solve(objective=objective, constraints=constraints, minimize=True)
hubs[0].solution_dict

{'BIG_M': 99999,
 'MAX_CARBON': None,
 'time': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 'technologies': ['Grid',
  'HP',
  'Boiler',
  'MicroCHP',
  'PV',
  'ST',
  'CHP',
  'GSHP',
  'Net_import',
  'Net_export'],
 'storages': ['Battery', 'Hot Water Tank'],
 'streams': ['Elec',
  'Heat',
  'Irradiation',
  'Grid',
  'Gas',
  'Net_import',
  'Net_export'],
 'output_streams': ['Elec', 'Heat', 'Net_export'],
 'demands': ['Elec', 'Heat'],
 'export_streams': ['Elec', 'Net_export'],
 'import_streams': ['Grid', 'Gas', 'Net_import'],
 'stream_timeseries': {'Irradiation': 'Irradiation'},
 'part_load': ['MicroCHP', 'CHP'],
 'sources': ['Irradiation'],
 'CONVERSION_EFFICIENCY': {'Grid': {'Elec': 1.0,
   'Heat': 0.0,
   'Irradiation': 0.0,
   'Grid': -1.0,
   'Gas': 0.0,
   'Net_import': 0.0,
   'Net_export': 0.0},
  'HP': {'Elec': -1.0,
   'Heat': 3.2,
   'Irradiation': 0.0,
   'Grid': 0.0,
   'Gas': 0.0,
   'Net_import': 0.0,
   'Net_export': 0.0},
  'Boiler': {'Elec': 0

### Solving the Models

Here we run the `multiple_hubs` function, which:
+ combines the individual hub models into one big hub
+ adds network constraints to allow energy to be exchanged
+ solves the resulting hub model  
The stdout is very long, so we capture it to a variable `sol_output`.

In [11]:
input_file = 'hub__'
n=4

In [12]:
%%capture sol_output
sol = multiple_hubs(input_files=input_file, n=n, network_excel='network.xlsx')

### Results

Results are returned for all four hubs, but all the network link capacities are the same by definition, so we can just take the values from the first hub.

In [13]:
hub = sol[0]

We can examine the network links that are installed:

In [14]:
hub["is_link_installed"]

{0: 1, 1: 1, 2: 0, 3: 1}

And the capacity of the links:

In [15]:
for i in range(4):
    id = 'capacity'+str(i)
    print('Link', i, 'has capacity', hub[id])

Link 0 has capacity 11.6479
Link 1 has capacity 8.16835
Link 2 has capacity 0.0
Link 3 has capacity 0.914739
